In [128]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow import keras
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Input, Concatenate


# 하이퍼파라미터 설정
image_height = 24
image_width = 32
batch_size = 32
epochs = 100


In [86]:
import os
import cv2
import json
import numpy as np
from collections import defaultdict
from sklearn.model_selection import train_test_split

# 데이터셋 경로 설정
dataset_dir = '/Users/mac/Dev/Project/ThermalCamera/Model/customModels/'
without_human_dir = os.path.join(dataset_dir, 'without_human')

# 이미지 파일 목록 읽기
without_human_images = [os.path.join(without_human_dir, filename) for filename in os.listdir(without_human_dir)]

test_dir = os.path.join(dataset_dir, 'test')
train_dir = os.path.join(dataset_dir, 'train')
valid_dir = os.path.join(dataset_dir, 'valid')

test_images = [os.path.join(test_dir, filename) for filename in os.listdir(test_dir)]
train_images = [os.path.join(train_dir, filename) for filename in os.listdir(train_dir)]
valid_images = [os.path.join(valid_dir, filename) for filename in os.listdir(valid_dir)]

print(test_images.__contains__('_groundtruth.json'))
print(train_images.__contains__('_groundtruth.json'))
print(valid_images.__contains__('_groundtruth.json'))
# print(valid_images)


# JSON 파일에서 이미지 이름 로드
test_json = os.path.join(test_dir, '_groundtruth.json')
with open(test_json, 'r') as f:
    test_image_names = json.load(f)

train_json = os.path.join(train_dir, '_groundtruth.json')
with open(train_json, 'r') as f:
    train_image_names = json.load(f)
    
valid_json = os.path.join(valid_dir, '_groundtruth.json')
with open(valid_json, 'r') as f:
    valid_image_names = json.load(f)


# for image_path in test_images:
#     count = 0
#     for item in test_image_names:
#         filename = item.get("filename")
#         if filename == os.path.basename(image_path):
#             print(os.path.basename(image_path))
#             count += 1
#     print(count)

# for image_path in train_images:
#     print(list(filter(lambda x:x["filename"]==os.path.basename(image_path),train_image_names)))
#     break;





False
False
False


In [129]:
data = []
numberOfPerson = []

# 'with_human' 이미지와 레이블(1) 추가
for image_path in train_images:
    if not image_path.endswith('.DS_Store') and not image_path.endswith('_groundtruth.json'):  # .DS_Store 파일 건너뛰기
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)  # 이미지를 흑백으로 읽어옴
        image = cv2.resize(image, (image_width, image_height))  # 이미지 크기 조절
        data.append(image)
        count = 0
        for item in train_image_names:
            filename = item.get("filename")
            if filename == os.path.basename(image_path):
                count += 1
        numberOfPerson.append(count)

for image_path in valid_images:
    if not image_path.endswith('.DS_Store') and not image_path.endswith('_groundtruth.json'):  # .DS_Store 파일 건너뛰기
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)  # 이미지를 흑백으로 읽어옴
        image = cv2.resize(image, (image_width, image_height))  # 이미지 크기 조절
        data.append(image)
        count = 0
        for item in valid_image_names:
            filename = item.get("filename")
            if filename == os.path.basename(image_path):
                count += 1
        numberOfPerson.append(count)
        
for image_path in test_images:
    if not image_path.endswith('.DS_Store') and not image_path.endswith('_groundtruth.json'):  # .DS_Store 파일 건너뛰기
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)  # 이미지를 흑백으로 읽어옴
        image = cv2.resize(image, (image_width, image_height))  # 이미지 크기 조절
        data.append(image)
        count = 0
        for item in test_image_names:
            filename = item.get("filename")
            if filename == os.path.basename(image_path):
                count += 1
        numberOfPerson.append(count)

# # 'without_human' 이미지와 레이블(0) 추가
# for image_path in without_human_images:
#     if not image_path.endswith('.DS_Store') and not image_path.endswith('_groundtruth.json'):  # .DS_Store 파일 건너뛰기
#         image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
#         image = cv2.resize(image, (image_width, image_height))
#         data.append(image)
#         numberOfPerson.append(0)


one_hot = tf.one_hot(numberOfPerson, 3)

# 데이터를 NumPy 배열로 변환
data = np.array(data, dtype=np.float32)
one_hot = np.array(one_hot, dtype = np.int32)
# 데이터를 0에서 1 사이의 값으로 정규화
data /= 255.0

# 첫 번째 호출로 훈련 데이터와 검증 데이터 생성
train_data, validation_data, train_numberOfPerson, validation_numberOfPerson = train_test_split(data, one_hot, test_size=0.1, random_state=42)

# 두 번째 호출로 테스트 데이터 생성 (훈련 데이터를 기반으로 생성하지 않음)
data_temp, test_data, numberOfPerson_temp, test_numberOfPerson = train_test_split(data, one_hot, test_size=0.1, random_state=42)

# 데이터의 shape 확인
print("Train data shape:", train_data.shape)
print("Validation data shape:", validation_data.shape)
print("Test data shape:", test_data.shape)

Train data shape: (2191, 24, 32)
Validation data shape: (244, 24, 32)
Test data shape: (244, 24, 32)


In [132]:
model = Sequential()

# Convolutional 레이어
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(image_height, image_width, 1)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

# Fully Connected 레이어
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))

# 출력 레이어
model.add(Dense(3, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [133]:
model.fit(
    train_data,  # 훈련 데이터
    train_numberOfPerson,  # 훈련 레이블
    epochs=epochs,  # 에포크 횟수
    validation_data=(validation_data, validation_numberOfPerson),  # 검증 데이터
    batch_size=batch_size  # 배치 크기
)


Epoch 1/100


2023-09-15 15:49:19.234150: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


69/69 [==============================] - ETA: 0s - loss: 0.6869 - accuracy: 0.6194

2023-09-15 15:49:26.923453: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


69/69 [==============================] - 10s 100ms/step - loss: 0.6869 - accuracy: 0.6194 - val_loss: 0.7687 - val_accuracy: 0.6148
Epoch 2/100
69/69 [==============================] - 4s 57ms/step - loss: 4.6685 - accuracy: 0.5755 - val_loss: 30.1474 - val_accuracy: 0.3279
Epoch 3/100
69/69 [==============================] - 4s 59ms/step - loss: 237.8934 - accuracy: 0.5212 - val_loss: 2667.8010 - val_accuracy: 0.6107
Epoch 4/100
69/69 [==============================] - 4s 56ms/step - loss: 7564.5771 - accuracy: 0.5436 - val_loss: 59493.3789 - val_accuracy: 0.6107
Epoch 5/100
69/69 [==============================] - 4s 57ms/step - loss: 104546.3984 - accuracy: 0.5299 - val_loss: 209264.0938 - val_accuracy: 0.3279
Epoch 6/100
69/69 [==============================] - 4s 56ms/step - loss: 679269.9375 - accuracy: 0.5253 - val_loss: 2754592.5000 - val_accuracy: 0.6107
Epoch 7/100
69/69 [==============================] - 4s 57ms/step - loss: 4001550.7500 - accuracy: 0.5367 - val_loss: 276712

KeyboardInterrupt: 

In [121]:
test_loss, test_accuracy = model.evaluate(test_data, test_numberOfPerson)
print(f"Test accuracy: {test_accuracy}")


8/8 [==============================] - 0s 46ms/step - loss: 31111405568.0000 - accuracy: 0.6107
Test accuracy: 0.6106557250022888


In [127]:
import cv2
import numpy as np

# 이미지 파일 경로 설정
img_path = '/Users/mac/Dev/Project/ThermalCamera/Model/customModels/test/2-_1-mp4_41_png.rf.1cd2ee1224fbccfe99d38870e28585b0.jpg'

# 이미지 읽기 및 전처리
image = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)  # 이미지를 흑백으로 읽어옴
image = cv2.resize(image, (image_width, image_height))  # 이미지 크기 조절
image = image.astype(np.float32) / 255.0  # 정규화

# 모델에 이미지 입력
predictions = model.predict(np.expand_dims(image, axis=0))  # 이미지를 배치 차원을 추가하여 모델에 전달

print(predictions)


1/1 [==============================] - 0s 15ms/step
[[0. 1. 0.]]


In [ ]:
model.save("detect.h5")  # 모델 저장